In [1]:
import subprocess
import docker
import os
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import time
import numpy as np
import time
from time import sleep
from tqdm.notebook import tqdm
from torch_geometric.utils import sort_edge_index
import torch
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
import gc

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def create_neo4j_container():
	client = docker.from_env()

	# Define the image name
	image_name = "neo4j:latest"

	# Define the container name and environment variables
	container_name = "neo4j-apoc"
	neo4j_env = {
		"NEO4J_apoc_export_file_enabled": "true",
		"NEO4J_apoc_import_file_enabled": "true",
		"NEO4J_apoc_import_file_use__neo4j__config": "true",
		"NEO4J_PLUGINS": '["apoc"]',
		"NEO4J_AUTH": "neo4j/password",
	}

	# Define the volume mapping
	volume_mapping = {
		f"{os.getcwd()}/neo4j_data": {"bind": "/data", "mode": "rw"},
		f"{os.getcwd()}/syn_data": {"bind": "/import", "mode": "rw"}
	}
	ports = {"7474": 7475, "7687": 7688}
	if len(client.containers.list(filters = {'name' : 'neo4j-apoc'})) == 0:
		container = client.containers.run(
			image=image_name,
			name=container_name,
			ports=ports,
			environment=neo4j_env,
			volumes=volume_mapping,
			detach=True,
			tty=True,
			stdin_open=True,
			remove=True,  # Automatically remove the container when it stops
		)
		sleep(5)
		print(f"Container {container_name} started with ID: {container.id}")
	else:
		print("Container neo4j-apoc already exists")
		container = client.containers.get('neo4j-apoc')

	max_timeout = 300  # Maximum wait time in seconds (5 minutes)
	start_time = time.time()

	uri = f"bolt://localhost:{ports['7687']}"
	username = "neo4j"
	password = "password"
	print("Waiting for Neo4j to start...")
	while True:
		try:
			driver = connect_to_neo4j(uri, username, password)
			driver.execute_query("MATCH (n) RETURN n")
			print("Neo4j is up and running.")
			return driver, container
		except Exception as e:
			print("Waiting for neo4j startup")
			if start_time > max_timeout:
				break
		sleep(5)
	return None, container

In [3]:
container, ports = create_neo4j_container()

Container neo4j-apoc started with ID: 6c342511b524a8a08fdc6367f6976aa01f577930d51ec21dde3cf246c4a58b0a
Waiting for Neo4j to start...
Waiting for neo4j startup


In [5]:
#container.stop()

In [4]:
def connect_to_neo4j(uri, user, password):
    """
    Establishes a connection to the Neo4j database.

    Args:
        uri (str): The URI of the Neo4j database.
        user (str): The username for authentication.
        password (str): The password for authentication.

    Returns:
        GraphDatabase.driver: The Neo4j driver instance for the connection.
    """

    return GraphDatabase.driver(uri, auth=(user, password))

In [5]:
# uri = f"bolt://localhost:{ports['7687']}"
uri = f"bolt://localhost:{7688}"
username = "neo4j"
password = "password"

driver = connect_to_neo4j(uri, username, password)
driver.execute_query("MATCH (n) DETACH DELETE n")

Transaction failed and will be retried in 0.9912263515758662s (Couldn't connect to localhost:7688 (resolved to ()):
Connection to 127.0.0.1:7688 closed without handshake response)
Transaction failed and will be retried in 2.0965165452019154s (Couldn't connect to localhost:7688 (resolved to ()):
Failed to read any data from server ResolvedIPv4Address(('127.0.0.1', 7688)) after connected (deadline Deadline(timeout=60.0)))


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7efdd04bf760>, keys=[])

In [19]:
driver.execute_query("MATCH (n) DETACH DELETE n")

Transaction failed and will be retried in 0.8479369836776482s (The allocation of an extra 1.5 GiB would use more than the limit 20.6 GiB. Currently using 19.2 GiB. dbms.memory.transaction.total.max threshold reached)


TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 1.5 GiB would use more than the limit 20.6 GiB. Currently using 19.2 GiB. dbms.memory.transaction.total.max threshold reached}

In [6]:
def recurse_edge_index_iterative(source_nodes, edge_index, max_depth):
    """
    Optimized function to compute the subgraph around the source nodes up to a given depth.
    Uses an iterative approach instead of recursion.
    """
    visited_nodes = set(source_nodes)
    current_frontier = np.array(source_nodes)
    
    subgraph_edges = []

    for _ in range(max_depth):
        # Find edges where the target node is in the current frontier
        target_mask = np.isin(edge_index[1], current_frontier)
        subgraph_edge_index = edge_index[:, target_mask]
        subgraph_edges.append(subgraph_edge_index)

        # Update the current frontier with the source nodes of these edges
        current_frontier = np.setdiff1d(subgraph_edge_index[0], list(visited_nodes))
        visited_nodes.update(current_frontier)
        
        if len(current_frontier) == 0:
            break

    # Combine edges from all hops
    return np.concatenate(subgraph_edges, axis=1) if subgraph_edges else np.empty((2, 0), dtype=edge_index.dtype)


def get_subgraph_from_in_mem_graph_optimized(X, y, i, edge_index, hops):
    """
    Optimized version of subgraph extraction.
    """
    subgraph_edge_index = recurse_edge_index_iterative([i], edge_index, hops)
    unique_node_ids, remapping = np.unique(subgraph_edge_index, return_inverse=True)
    
    # Extract features and labels
    features = X.iloc[unique_node_ids, :].values
    labels = y.iloc[unique_node_ids, :].values.squeeze()

    # Remap edge indices
    remapped_edge_index = remapping.reshape(2, -1)
    return remapped_edge_index, features, labels, unique_node_ids
    
def create(driver, X_and_y, node_file_name, edge_file_name):
    label_columns = list(filter(lambda col: "y" in col, X_and_y.columns))
    feature_columns = list(filter(lambda col: "f" in col, X_and_y.columns))
    label_columns_setter = "\n".join([f"SET p['{col}'] = toInteger(line['{col}'])" for col in label_columns])
    feature_columns_setter = "\n".join([f"SET p['{col}'] = toFloat(line['{col}'])" for col in feature_columns])
    with driver.session() as session:
        start = time.time()
        node_query = f"""
                LOAD CSV WITH HEADERS FROM $file AS line
                WITH line, linenumber() AS index
                CALL (line, index) {{
                  MERGE (p:Node {{id: index - 2}})
                  {label_columns_setter}
                  {feature_columns_setter}
                }} IN TRANSACTIONS OF 10000 ROWS
                """
        session.run(node_query, file=f"file:///{node_file_name}")
        session.run("CREATE INDEX IF NOT EXISTS FOR (n:Node) ON (n.id)")
        edge_query = """
            LOAD CSV FROM $file AS line
            WITH line, linenumber() AS index
            WHERE index <> 1
            CALL (line) {
                MATCH (s:Node {id: toInteger(line[0])}), (t:Node {id: toInteger(line[1])})
                CREATE (s)-[r:connects]->(t)
            } IN TRANSACTIONS OF 10000 ROWS
            """
        session.run(edge_query, file=f"file:///{edge_file_name}")
        return time.time() - start
        
def read(driver, X_and_y, hops, X, y, edge_index, random_sample_size= 1000):
    overall_run_time = 0     
    test_time = 0     
    label_columns = list(filter(lambda col: "y" in col, X_and_y.columns))
    feature_columns = list(filter(lambda col: "f" in col, X_and_y.columns))
    feature_columns_getter = ",".join([f"node['{col}']" for col in feature_columns])
    feature_columns_getter = f'[{feature_columns_getter}] as nodeFeatures'
    label_columns_getter = ",".join([f"node['{col}']" for col in label_columns])
    label_columns_getter = f'[{label_columns_getter}] as nodeLabels'
    with driver.session(database = "neo4j") as session:
        np.random.seed(42)
        seed_node_ids = np.random.choice(np.arange(X_and_y.shape[0]), size = random_sample_size, replace = False)
        for seed_node_id in tqdm(seed_node_ids):
            start = time.time()
            subgraph_query = f"""
                MATCH (t {{id: $seed_node_id}})
                MATCH (s)-[r*0..{hops}]->(t)
                UNWIND r AS edge
                WITH t, edge
                WITH 
                  collect(DISTINCT [startNode(edge).id, endNode(edge).id]) AS edges,
                  collect(DISTINCT startNode(edge)) AS startNodes,
                  t AS endNode
                UNWIND (startNodes + [endNode]) AS node
                WITH edges, COLLECT(DISTINCT node) AS uniqueNodes
                UNWIND uniqueNodes AS node
                WITH edges, node.id as nodeId, {feature_columns_getter}, {label_columns_getter}
                ORDER BY nodeId
                RETURN 
                  edges,
                  collect(nodeId) AS idCollection,
                  collect(nodeLabels) AS labels,
                  collect(nodeFeatures) AS features
                """
            ## ORDER BY nodeId seems to increase speed 
            results = session.run(subgraph_query, seed_node_id = seed_node_id)
            res_df = results.to_df()
            if res_df.empty: continue
            subgraph_edge_index = np.array(res_df["edges"][0]).transpose()        
            node_ids = np.array(res_df["idCollection"][0])
            # id_sort_idx = np.argsort(node_ids)
            # node_ids = node_ids[id_sort_idx]
            cols_source = np.searchsorted(node_ids, subgraph_edge_index[0])
            cols_target = np.searchsorted(node_ids, subgraph_edge_index[1])
            
            remapped_edge_index = np.concatenate([np.expand_dims(cols_source, axis = 0), np.expand_dims(cols_target, axis = 0)], axis = 0)
            features =np.array(res_df["features"][0]) #[id_sort_idx]
            labels = np.array(res_df["labels"][0]) #[id_sort_idx]   
            overall_run_time += time.time() - start    
            
            ## Testing
            start = time.time()
            remapped_edge_index_test, features_test, labels_test, unique_node_ids = get_subgraph_from_in_mem_graph_optimized(X, y, seed_node_id, edge_index, hops)        
            test_time += time.time() - start
            assert (sort_edge_index(torch.from_numpy(remapped_edge_index_test)) == sort_edge_index(torch.from_numpy(remapped_edge_index))).sum() / (remapped_edge_index_test.shape[-1] * remapped_edge_index_test.shape[0]), "Edges doesnt match"
            del remapped_edge_index_test
            del remapped_edge_index
            gc.collect()
            assert np.allclose(node_ids, unique_node_ids), "Node ids does not match"
            del node_ids
            del unique_node_ids
            gc.collect()
            assert np.allclose(features.astype(np.float32), features_test.astype(np.float32)), "features does not match"
            del features
            del features_test
            gc.collect()
            assert np.allclose(labels_test, labels), "Labels does not match"
            del labels
            del labels_test
            gc.collect()
        return overall_run_time, test_time

def update_nodes(driver, X_and_y, random_sample_size = 1000):
    feature_columns = list(filter(lambda col: "f" in col, X_and_y.columns))
    label_columns = list(filter(lambda col: "y" in col, X_and_y.columns))
    with driver.session(database = "neo4j") as session:
        np.random.seed(42)
        node_ids = np.random.choice(np.arange(X_and_y.shape[0]), size = random_sample_size, replace = False).tolist()
        overall_run_time = 0
        for node_id in tqdm(node_ids):
            np.random.seed(42)
            features = np.random.rand(len(feature_columns)).tolist()  # Random values between 0 and 1
            labels = np.random.randint(0, 2, size=len(label_columns)).tolist()  # Adjust label range as needed  
            label_columns_setter = "\n".join([f"SET n['{col}'] = {int(labels[i])}" for i, col in enumerate(label_columns)])
            feature_columns_setter = "\n".join([f"SET n['{col}'] = {features[i]}" for i, col in enumerate(feature_columns)])
            start = time.time()
            cypher_query = f"""
            MATCH (n:Node {{id: $node_id}})
            {label_columns_setter}
            {feature_columns_setter}
            """
            session.run(cypher_query, node_id=node_id)
            overall_run_time += time.time() - start
        return overall_run_time

def update_edges(driver, edge_index, X_and_y, random_sample_size = 1000):
    np.random.seed(42)
    edge_ids = np.random.choice(np.arange(edge_index.shape[-1]),
                                size=random_sample_size,
                                replace=False).tolist()
    selected_edges = edge_index[:, edge_ids].transpose(-1, 0)
    with driver.session() as session:
        start = time.time()
        for selected_edge in tqdm(selected_edges, desc="Updating edges"):
            source_id, target_id = selected_edge
            np.random.seed(42)
            new_target_id = int(np.random.randint(0, X_and_y.shape[0]))
            
            # Cypher query to update the target_id of the relationship that matches both source_id and target_id.
            cypher_query = """
            MATCH (s {id: $source_id}), (t {id: $target_id}), (new_t {id: $new_target_id})
            MATCH (s)-[r:connects]->(t)
            DELETE r
            CREATE (s)-[new_r:connects]->(new_t)
            """
            session.run(cypher_query,
                        source_id=int(source_id),
                        target_id=int(target_id),
                        new_target_id=new_target_id)
    
    return time.time() - start
    

def delete(driver):
    start = time.time()
    driver.execute_query("MATCH (n) DETACH DELETE(n)")
    return time.time() - start

In [7]:
edge_file_name = "ppi_edge_index.csv"
node_file_name = "ppi.csv"
X = pd.read_csv(f"data/ppi_x.csv")
y = pd.read_csv(f"data/ppi_y.csv")
edges = pd.read_csv("data/" + edge_file_name)
edges.columns = ["source_id", "target_id"]

X.columns = list(map(lambda col: f"f_{col}", X.columns))
y.columns = list(map(lambda col: f"y_{col}", y.columns))
y = y.astype(np.int8)
X_and_y = X.copy()
X_and_y = pd.concat((X_and_y, y), axis = 1)

node_file_name = "X_y_ppi_col.csv" ## Can use the same structure for all database when using column wise import
X_and_y.to_csv(f"syn_data/{node_file_name}", sep = ",", index = True)
edges.to_csv(f"syn_data/{edge_file_name}", sep = ",", index = False)
edge_index = edges.values.transpose(-1, 0)
create_time = create(driver, X_and_y, node_file_name, edge_file_name)
read_times = dict()
read_times_mem = dict()
for hops in tqdm(range(1, 4)):
    read_time, read_time_mem = read(driver, X_and_y, hops, X, y, edge_index, random_sample_size= 1000)
    read_times[hops] = read_time
    read_times_mem[hops] = read_time_mem
update_time_nodes = update_nodes(driver, X_and_y, random_sample_size = 1000)
update_time_edges = update_edges(driver, edge_index, X_and_y, random_sample_size = 1000)
delete_time = delete(driver)
new_row_dict = {"name": "PPI", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
for hops in read_times:
    new_row_dict[f"read_{hops}"] = read_times[hops]
    new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
new_row = pd.DataFrame([new_row_dict])
output_df = pd.concat((output_df, new_row), ignore_index=True)


KeyboardInterrupt



In [30]:
def eval_ppi(output_df):
    edge_file_name = "ppi_edge_index.csv"
    node_file_name = "ppi.csv"
    X = pd.read_csv(f"data/ppi_x.csv")
    y = pd.read_csv(f"data/ppi_y.csv")
    edges = pd.read_csv("data/" + edge_file_name)
    edges.columns = ["source_id", "target_id"]

    X.columns = list(map(lambda col: f"f_{col}", X.columns))
    y.columns = list(map(lambda col: f"y_{col}", y.columns))
    y = y.astype(np.int8)
    X_and_y = X.copy()
    X_and_y = pd.concat((X_and_y, y), axis = 1)
    
    node_file_name = "X_y_ppi_neo4j_col.csv" ## Can use the same structure for all database when using column wise import
    X_and_y.to_csv(f"syn_data/{node_file_name}", sep = ",", index = True)
    edges.to_csv(f"syn_data/{edge_file_name}", sep = ",", index = False)
    edge_index = edges.values.transpose(-1, 0)
    create_time = create(driver, X_and_y, node_file_name, edge_file_name)
    read_times = dict()
    read_times_mem = dict()
    for hops in tqdm(range(1, 4)):
        read_time, read_time_mem = read(driver, X_and_y, hops, X, y, edge_index, random_sample_size= 1000)
        read_times[hops] = read_time
        read_times_mem[hops] = read_time_mem
    update_time_nodes = update_nodes(driver, X_and_y, random_sample_size = 1000)
    update_time_edges = update_edges(driver, edge_index, X_and_y, random_sample_size = 1000)
    delete_time = delete(driver)
    new_row_dict = {"name": "PPI", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
    for hops in read_times:
        new_row_dict[f"read_{hops}"] = read_times[hops]
        new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
    new_row = pd.DataFrame([new_row_dict])
    output_df = pd.concat((output_df, new_row), ignore_index=True)
    return output_df
	
def eval_synth(output_df):
    for num_nodes in tqdm([1_000, 10_000, 100_000, 1_000_000]):
        for num_edges in tqdm(["5_edges", "10_edges", "20_edges", "scale_free"]):
            if num_nodes == 1_000_000 and (num_edges == "10_edges" or num_edges == "20_edges"): continue
            feature_file_name = f"X_{str(num_nodes)}_nodes_{num_edges}.csv"
            label_file_name = f"y_{str(num_nodes)}_nodes_{num_edges}.csv"
            edge_file_name = f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv"
            assert os.path.exists(f"syn_data/{feature_file_name}"), "Feature file does not exist"
            assert os.path.exists(f"syn_data/{label_file_name}"), "Label file does not exist"
            assert os.path.exists(f"syn_data/{edge_file_name}"), "Edge file does not exist"
            
            X = pd.read_csv(f"syn_data/{feature_file_name}")
            X.columns = list(map(lambda col: f"f_{col}", X.columns))
            y = pd.read_csv(f"syn_data/{label_file_name}")
            y.columns = list(map(lambda col: f"y_{col}", y.columns))
            y = y.astype(np.int8)
            
            edges = pd.read_csv(f"syn_data/{edge_file_name}")
            edges.columns = ["source_id", "target_id"]
            edge_index = edges.values.transpose(-1, 0)
            node_file_name = f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_neo4j_col.csv"
            if True or not os.path.exists(f"syn_data/{node_file_name}"):
                X_and_y = X.copy()
                X_and_y = pd.concat((X_and_y, y), axis = 1)
                X_and_y.to_csv(f"syn_data/{node_file_name}", sep = ",", index = True)
            else:
                X_and_y = pd.read_csv(f"syn_data/{node_file_name}", sep = ";", index_col = 0)
            create_time = create(driver, X_and_y, node_file_name, edge_file_name)
            read_times = dict()
            read_times_mem = dict()
            for hops in tqdm(range(1, 4)):
                read_time, read_time_mem = read(driver, X_and_y, hops, X, y, edge_index, random_sample_size= 1000)
                read_times[hops] = read_time
                read_times_mem[hops] = read_time_mem
            update_time_nodes = update_nodes(driver, X_and_y, random_sample_size = 1000)
            update_time_edges = update_edges(driver, edge_index, X_and_y, random_sample_size = 1000)
            delete_time = delete(driver)
            new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_time_nodes, "update_edges": update_time_edges, "delete": delete_time}
            for hops in read_times:
                new_row_dict[f"read_{hops}"] = read_times[hops]
                new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
            new_row = pd.DataFrame([new_row_dict])
            output_df = pd.concat((output_df, new_row), ignore_index=True)
    return output_df

In [31]:
num_iterations = 1
off_set = 0
for i in range(off_set, num_iterations + off_set):
    print(f"Iteration {i}")
    output_df = pd.DataFrame(columns = ["name", "create", "update_nodes", "update_edges", "delete"])
    output_df = eval_ppi(output_df)
    output_df = eval_synth(output_df)
    output_df.to_csv(f"neo4j_col_{i}.csv")

Iteration 0


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [9]:
for num_nodes in tqdm([1_000, 10_000, 100_000, 1_000_000]):
    for num_edges in tqdm(["5_edges", "10_edges", "20_edges", "scale_free"]):
        if num_nodes == 1_000_000 and (num_edges == "10_edges" or num_edges == "20_edges"): continue
        feature_file_name = f"X_{str(num_nodes)}_nodes_{num_edges}.csv"
        label_file_name = f"y_{str(num_nodes)}_nodes_{num_edges}.csv"
        edge_file_name = f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv"
        assert os.path.exists(f"syn_data/{feature_file_name}"), "Feature file does not exist"
        assert os.path.exists(f"syn_data/{label_file_name}"), "Label file does not exist"
        assert os.path.exists(f"syn_data/{edge_file_name}"), "Edge file does not exist"
        
        X = pd.read_csv(f"syn_data/{feature_file_name}")
        X.columns = list(map(lambda col: f"f_{col}", X.columns))
        y = pd.read_csv(f"syn_data/{label_file_name}")
        y.columns = list(map(lambda col: f"y_{col}", y.columns))
        y = y.astype(np.int8)
        
        edges = pd.read_csv(f"syn_data/{edge_file_name}")
        edges.columns = ["source_id", "target_id"]
        edge_index = edges.values.transpose(-1, 0)
        node_file_name = f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_neo4j_col.csv"
        if True or not os.path.exists(f"syn_data/{node_file_name}"):
            X_and_y = X.copy()
            X_and_y = pd.concat((X_and_y, y), axis = 1)
            X_and_y.to_csv(f"syn_data/{node_file_name}", sep = ",", index = True)
        else:
            X_and_y = pd.read_csv(f"syn_data/{node_file_name}", sep = ";", index_col = 0)
        create_time = create(driver, X_and_y, node_file_name, edge_file_name)
        read_times = dict()
        read_times_mem = dict()
        for hops in tqdm(range(1, 4)):
            read_time, read_time_mem = read(driver, X_and_y, hops, X, y, edge_index, random_sample_size= 1000)
            read_times[hops] = read_time
            read_times_mem[hops] = read_time_mem
        update_time_nodes = update_nodes(driver, X_and_y, random_sample_size = 1000)
        update_time_edges = update_edges(driver, edge_index, X_and_y, random_sample_size = 1000)
        delete_time = delete(driver)
        new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_time_nodes, "update_edges": update_time_edges, "delete": delete_time}
        for hops in read_times:
            new_row_dict[f"read_{hops}"] = read_times[hops]
            new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
        new_row = pd.DataFrame([new_row_dict])
        output_df = pd.concat((output_df, new_row), ignore_index=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
output_df

,name,create,update_nodes,update_edges,delete,read_1,read_in_mem_1,read_2,read_in_mem_2,read_3,read_in_mem_3
0,1000_nodes_5_edges,0.724191,14.620965,20.088695,0.094668,23.198426,1.722600,47.961281,2.490153,84.894798,1.766927
1,1000_nodes_10_edges,0.669602,13.031957,19.974057,0.125455,27.695162,1.822084,73.798971,1.832818,217.004385,1.223876
2,1000_nodes_20_edges,0.491300,11.835071,17.506212,0.183648,37.931068,2.024508,119.818688,1.113013,539.471215,2.192494
3,1000_nodes_scale_free,0.330520,13.361930,19.321021,0.081727,4.575462,0.376970,9.635729,0.439245,23.244678,0.415799
4,10000_nodes_5_edges,3.473168,12.913085,66.346031,0.941029,36.675515,2.400790,57.512045,5.750876,98.885532,3.977346
5,10000_nodes_10_edges,4.557181,13.062772,64.708819,1.166279,40.795364,3.255892,86.670575,5.034676,349.527975,4.146428
6,10000_nodes_20_edges,5.495414,13.799639,65.100739,1.766707,49.201019,4.981297,156.750628,4.114111,2083.930239,7.910517
7,10000_nodes_scale_free,3.154809,12.532086,64.797487,0.873238,9.486132,0.465755,28.702118,0.667932,103.862419,0.805704
8,100000_nodes_5_edges,37.576545,13.858868,560.523144,13.924678,281.266107,10.219358,293.310896,29.860370,331.460629,23.441089
9,100000_nodes_10_edges,50.234440,12.937322,561.987473,16.734241,285.107854,18.198174,317.870894,28.395054,614.013816,26.089577
